In [ ]:
from pathlib import Path
import re
from PIL import Image

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from unet import iou_loss, dice_loss, bce_loss

# get binary crossentropy loss
bce = tf.keras.losses.BinaryCrossentropy()

In [ ]:
# load a model
print(list(Path("../models/").iterdir()))
model = tf.keras.models.load_model(
    "../models/catsnet_model.h5", custom_objects={"iou_loss": iou_loss, "dice_loss": dice_loss}
)

In [ ]:
seed = 0
np.random.seed(seed)

data_dir = Path("../data/all_data/")
assert data_dir.exists()

# Find the indexes of all the image files in the format of image_<index>.npy
image_indexes = [int(re.search(r"\d+", file.name).group()) for file in data_dir.glob("image_*.npy")]
mask_indexes = [int(re.search(r"\d+", file.name).group()) for file in data_dir.glob("mask_*.npy")]

# Get a test dataset of 0.2 * size of the dataset
test_size = int(0.2 * len(image_indexes))
print(f"Test size: {test_size}")

# Randomly select test_size number of indexes
test_indexes = np.random.choice(image_indexes, test_size, replace=False)
print(f"Test indexes: {test_indexes}")

In [ ]:
def iou_loss_simple(y_true, y_pred, smooth=1e-5):
    """Intersection over Union loss function for a single image."""

    intersection = tf.reduce_sum(y_true * y_pred)
    union = tf.reduce_sum(y_true + y_pred) - intersection
    return 1 - (intersection + smooth) / (union + smooth)


# test it

ground_truth = np.zeros((10, 10))
# ground_truth[2:8, 2:8] = 1
ground_truth[5, 5] = 1
plt.imshow(ground_truth, cmap="gray")
plt.show()

prediction = np.zeros((10, 10))
# prediction[3:7, 3:7] = 1
prediction[5, 5] = 1
plt.imshow(prediction, cmap="gray")
plt.show()

print(iou_loss(ground_truth, prediction))

In [ ]:
def iou_loss(y_true, y_pred, smooth=1e-5):
    """Intersection over Union loss function.

    Parameters
    ----------
    y_true : tf.Tensor
        True values.
    y_pred : tf.Tensor
        Predicted values.
    smooth : float
        Smoothing factor to prevent division by zero.

    Returns
    -------
    iou : tf.Tensor
        The IoU loss.
    """
    # Ensure the tensors are of the same shape
    y_true = tf.squeeze(y_true, axis=-1) if y_true.shape[-1] == 1 else y_true
    y_pred = tf.squeeze(y_pred, axis=-1) if y_pred.shape[-1] == 1 else y_pred
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)

    intersection = tf.reduce_sum(y_true * y_pred, axis=(1, 2))
    sum_of_squares_pred = tf.reduce_sum(tf.square(y_pred), axis=(1, 2))
    sum_of_squares_true = tf.reduce_sum(tf.square(y_true), axis=(1, 2))
    # print(f"intersection: {intersection} union: {sum_of_squares_pred + sum_of_squares_true - intersection}")
    iou = 1 - (intersection + smooth) / (sum_of_squares_pred + sum_of_squares_true - intersection + smooth)
    # return the mean over all samples in the batch, as a single value
    return float(tf.reduce_mean(iou))


# test it

ground_truth = np.zeros((10, 10))
# ground_truth[2:8, 2:8] = 1
ground_truth[5, 5:7] = 1
plt.imshow(ground_truth, cmap="gray")
plt.show()

prediction = np.zeros((10, 10))
# prediction[3:7, 3:7] = 1
prediction[5, 5] = 1
plt.imshow(prediction, cmap="gray")
plt.show()

ground_truth = np.expand_dims(ground_truth, axis=0)
prediction = np.expand_dims(prediction, axis=0)

print(iou_loss(ground_truth, prediction))

In [ ]:
dice_totals = 0.0
iou_totals = 0.0
binary_crossentropy_totals = 0.0

for test_index in test_indexes:

    # Load image and mask
    image = np.load(data_dir / f"image_{test_index}.npy")
    mask = np.load(data_dir / f"mask_{test_index}.npy")

    # Resize the image and mask to the model image size
    pil_image = Image.fromarray(image)
    pil_image = pil_image.resize((512, 512))
    image = np.array(pil_image).astype(np.float32)
    pil_mask = Image.fromarray(mask)
    pil_mask = pil_mask.resize((512, 512))
    mask = np.array(pil_mask).astype(np.float32)

    # Add batch dimension
    image = np.expand_dims(image, axis=0)
    mask = np.expand_dims(mask, axis=0)
    # Add channel dimension
    image = np.expand_dims(image, axis=-1)
    mask = np.expand_dims(mask, axis=-1)

    # Predict the mask
    predicted_mask = model.predict(image)

    # print(f"predicted mask min max: {np.min(predicted_mask)}, {np.max(predicted_mask)}")

    # plt.imshow(predicted_mask[0, :, :, 0])
    # plt.title("Predicted Mask")
    # plt.show()
    # plt.imshow(mask[0, :, :, 0])
    # plt.title("Ground Truth Mask")
    # plt.show()
    # plt.imshow(image[0, :, :, 0])
    # plt.title("Image")
    # plt.show()

    # Calculate the dice loss
    dice_loss_value = dice_loss(mask, predicted_mask)
    dice_totals += dice_loss_value

    # Calculate the iou loss
    iou_loss_value = iou_loss(mask, predicted_mask)
    iou_totals += iou_loss_value

    # Calculate the binary crossentropy loss
    binary_crossentropy_loss_value = tf.reduce_mean(tf.keras.losses.binary_crossentropy(mask, predicted_mask))
    binary_crossentropy_totals += binary_crossentropy_loss_value.numpy()

    print(
        f"Index {test_index} Losses: dice: {dice_loss_value}, iou: {iou_loss_value}, binary_crossentropy: {binary_crossentropy_loss_value}"
    )

# Calculate the average losses
average_dice_loss = dice_totals / test_size
average_iou_loss = iou_totals / test_size
average_binary_crossentropy_loss = binary_crossentropy_totals / test_size

print(
    f"Average Losses: dice: {average_dice_loss}, iou: {average_iou_loss}, binary_crossentropy: {average_binary_crossentropy_loss}"
)

In [ ]:
model.summary()